# Блочная нормализация
Одним из путей повышения эффективности обучения в глубоких сетях является использование более сложных процедур оптимизации, таких как SGD с моментом, RMSProp или Adam. Другая стратегия - изменить архитектуру сети, чтобы улучшить ее обучаемость. Одной из идей в этой области является блочная нормализация, которая была предложена в [3] в 2015 году.

Идея относительно проста. Методы машинного обучения имеют тенденцию работать лучше, когда их входные данные состоят из некоррелированных признаков с нулевым средним и единичной дисперсией. При обучении нейронной сети мы можем предварительно обработать данные, прежде чем загружать их в сеть, чтобы явно декореллировать признаки; это гарантирует, что на первый уровень сети поступают данные, которые характеризуются хорошим распределением. Однако, даже если мы предварительно обрабатываем входные данные, выходы более глубоких слоев сети, скорее всего, больше не будут декореллированы и не будут иметь нулевое среднее и единичную дисперсию. Еще хуже то, что во время обучения распределение признаков на каждом уровне сети будет меняться по мере обновления весов каждого уровня.

Авторы [3] предполагают, что смещение распределения прзнаков внутри глубоких нейронных сетей затрудняет обучение. Чтобы преодолеть это, [3] предлагается вставить в сеть слои блочной нормализации. Во время обучения слой блочной нормализации использует мини-блок данных для оценки среднего и стандартного отклонения каждого признака. Эти средние затем используются для центрирования и нормализации признаков на  мини-блоке в ходе обучения. Текущее среднее и текущее стандартное отклонение выичсляемые во время обучения можно сохранять, а во время тестирования использовать их для центрирования и нормализации признаков.

Возможно, что эта стратегия нормализации снижает репрезентативную емкость сети, так как иногда для некоторых слоев оптимальными будут признаки, которые не обладают нулевым средним и единичной дисперсией. С этой целью слой блочной нормализации включает в себя обучаемые параметры смещения и масштаба для каждого признака.

[3] [Sergey Ioffe and Christian Szegedy, "Batch Normalization: Accelerating Deep Network Training by Reducing
Internal Covariate Shift", ICML 2015.](https://arxiv.org/abs/1502.03167)

In [ ]:
# Выполните начальные установки блокнота.
# Внимание! Блокнот работает в версии Python 3.6!

import time
import numpy as np
import matplotlib.pyplot as plt
from dlcv.classifiers.fc_net import *
from dlcv.data_utils import get_CIFAR10_data
from dlcv.gradient_check import eval_numerical_gradient, eval_numerical_gradient_array
from dlcv.solver import Solver

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # установка размеров графиков по умолчанию
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Для перезагрузки внешних модулей python;
# см. http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

def rel_error(x, y):
    """ возвращает относительную ошибку """
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

def print_mean_std(x,axis=0):
    print('  среднее: ', x.mean(axis=axis))
    print('  стандартное отклонение:  ', x.std(axis=axis))
    print() 

In [ ]:
# Загрузка и предобработка данных CIFAR10

data = get_CIFAR10_data()
for k, v in data.items():
  print('%s: ' % k, v.shape)

## Блочная нормализация: прямое распространение
В файле `dlcv/layers.py` реализуйте функцию блочной нормализации  `batchnorm_forward` для прямого пути (возможно Вам повезло и функция уже написана). Затем выполните код ниже, чтобы проверить реализацию.

In [ ]:
# Проверьте прямое распространение для режима обучения, проверив средние значения и дисперсии
# признаков как до, так  и после блочной нормализации

# Моделируем прямое распросранение для 2-х слойной сети
np.random.seed(231)
N, D1, D2, D3 = 200, 50, 60, 3
X = np.random.randn(N, D1)
W1 = np.random.randn(D1, D2)
W2 = np.random.randn(D2, D3)
a = np.maximum(0, X.dot(W1)).dot(W2)

print('Перед блочной нормализацией:')
print_mean_std(a,axis=0)

gamma = np.ones((D3,))
beta = np.zeros((D3,))

# Средние значения должны быть близки к нулю, а стандартные отклонения – к единице.
print('После блочной нормализации (gamma=1, beta=0)')
a_norm, _ = batchnorm_forward(a, gamma, beta, {'mode': 'train'})
print_mean_std(a_norm,axis=0)

gamma = np.asarray([1.0, 2.0, 3.0])
beta = np.asarray([11.0, 12.0, 13.0])

# Теперь средние значения должны быть близки к beta, а стандартные отклонения — к gamma.
print('После блочной нормализации (gamma=', gamma, ', beta=', beta, ')')
a_norm, _ = batchnorm_forward(a, gamma, beta, {'mode': 'train'})
print_mean_std(a_norm,axis=0)

In [ ]:
# Проверяем прямое распространение через слой BN в режиме тестирования, запустив прямое распростанение 
# в режиме обучения много раз, чтобы накопить текущие средние значения, а затем
# проверяем средние и дисперсии на выходе слоя блочной нормализации в режиме тестирования

np.random.seed(231)
N, D1, D2, D3 = 200, 50, 60, 3
W1 = np.random.randn(D1, D2)
W2 = np.random.randn(D2, D3)

bn_param = {'mode': 'train'}
gamma = np.ones(D3)
beta = np.zeros(D3)

for t in range(50):
  X = np.random.randn(N, D1)
  a = np.maximum(0, X.dot(W1)).dot(W2)
  batchnorm_forward(a, gamma, beta, bn_param)

bn_param['mode'] = 'test'
X = np.random.randn(N, D1)
a = np.maximum(0, X.dot(W1)).dot(W2)
a_norm, _ = batchnorm_forward(a, gamma, beta, bn_param)

# Средние должны быть близки к нулю, а стандартные отклонения к 1,
# но будут больше зашумлены, чем в режиме обучения
print('После блочной нормализации (режим тестирования):')
print_mean_std(a_norm,axis=0)

## Блочная нормализация: обратное рапространение
Теперь реализуем обратное распространение для  блочной нормализации в функции `batchnorm_backward`.

Чтобы реализовать обратный путь, вы должны представить граф вычислений для блочной нормализации  и обратное распространение через каждый из промежуточных узлов. Некоторые промежуточные узлы могут иметь несколько исходящих ветвей; обязательно суммируйте градиенты по этим ветвям на  обратном пути.

После реализации функции (возможно Вам повезло и функция уже написана), запустите код ниже, чтобы проверить реализацию обратного распространения.

In [ ]:
# Проверка градиентов при обратном распространении через слой блочной нормализации
np.random.seed(231)
N, D = 4, 5
x = 5 * np.random.randn(N, D) + 12
gamma = np.random.randn(D)
beta = np.random.randn(D)
dout = np.random.randn(N, D)

bn_param = {'mode': 'train'}
fx = lambda x: batchnorm_forward(x, gamma, beta, bn_param)[0]
fg = lambda a: batchnorm_forward(x, a, beta, bn_param)[0]
fb = lambda b: batchnorm_forward(x, gamma, b, bn_param)[0]

dx_num = eval_numerical_gradient_array(fx, x, dout)
da_num = eval_numerical_gradient_array(fg, gamma.copy(), dout)
db_num = eval_numerical_gradient_array(fb, beta.copy(), dout)

_, cache = batchnorm_forward(x, gamma, beta, bn_param)
dx, dgamma, dbeta = batchnorm_backward(dout, cache)
# Относительная ошибка должна быть в пределах 1e-13 and 1e-8
print('Ошибка градиента dx: ', rel_error(dx_num, dx))
print('Ошибка градиента dgamma: ', rel_error(da_num, dgamma))
print('Ошибка градиента dbeta: ', rel_error(db_num, dbeta))

## Блочная нормализация: альтернативное распространение
При реализации обратного распространения используют два подхода. Первый подход заключается в том, чтобы представить граф вычислений в виде простых операций над всеми промежуточными значениями и при построении обратного пути использовать производные для этих элементарных операций. Второй подход заключается в использовании аналитических производных более крупных операций. Например, вы можете получить очень простую формулу для обратного распространения градиентов через сигмовидную функцию.

Удивительно, но оказывается, что вы можете сделать аналогичное упрощение и для обратного пути в случае блочной нормализации. При заданном множестве входов $X=\begin{bmatrix}x_1\\x_2\\...\\x_N\end{bmatrix}$, 
мы сначала вычисляем среднее $\mu=\frac{1}{N}\sum_{k=1}^N x_k$ и дисперсию $v=\frac{1}{N}\sum_{k=1}^N (x_k-\mu)^2.$    
При вычисленных $\mu$ and $v$ , мы можем вычислить стандартное отклонение $\sigma=\sqrt{v+\epsilon}$  и нормализовать данные $Y$ с помощью $y_i=\frac{x_i-\mu}{\sigma}.$

<img src="batchnorm_graph.png">

Решить нашу проблему - это получить $\frac{\partial L}{\partial X}$  из восходящего градиента $\frac{\partial L}{\partial Y}$. Чтобы сделать это, вспомните цепочное правило. 
Возможно, было бы сложно прямо рассуждать о градиентах над $X$ и $Y$ - попробуйте рассуждать об этом в терминах $x_i$ и $y_i$ сначала.

Вам нужно будет придумать вывод для $\frac{\partial L}{\partial x_i}$, полагаясь на цепочное правило, чтобы сначала вычислить промежуточные градиенты $\frac{\partial \mu}{\partial x_i}, \frac{\partial v}{\partial x_i}, \frac{\partial \sigma}{\partial x_i},$ затем собрать эти части для вычисления  $\frac{\partial y_i}{\partial x_i}$. Вы должны убедиться, что каждый из промежуточных шагов максимально прост.

После этого выполните проверку альтернативной блочной нормализации с помощью функции `batchnorm_backward_alt` и сравните две реализации, выполнив код ниже (возможно Вам повезло и функция уже написана, тогда просто проанализируйте её код). Ваши две реализации обратного распространения должны возвращать почти одинаковые результаты, но альтернативная реализация должна быть немного быстрее.

In [ ]:
np.random.seed(231)
N, D = 100, 500
x = 5 * np.random.randn(N, D) + 12
gamma = np.random.randn(D)
beta = np.random.randn(D)
dout = np.random.randn(N, D)

bn_param = {'mode': 'train'}
out, cache = batchnorm_forward(x, gamma, beta, bn_param)

#t1 = time.time()
dx1, dgamma1, dbeta1 = batchnorm_backward(dout, cache)
#t2 = time.time()
dx2, dgamma2, dbeta2 = batchnorm_backward_alt(dout, cache)
#t3 = time.time()

print('Отличие в dx : ', rel_error(dx1, dx2))
print('Отличие в dgamma: ', rel_error(dgamma1, dgamma2))
print('Отличие в dbeta: ', rel_error(dbeta1, dbeta2))
#print('Время выполнения для обычной нормализации : %.50fx' % (t2 - t1))
#print('Время выполнения для альтернативной нормализации : %.50fx' %  (t3 - t2))

## Полносвязанные сети с блочной нормализацией

Теперь, когда у вас есть рабочая реализация блочной нормализации, вернитесь к  классу FullyConnectedNet в файле dlcv/classifiers/fc_net.py. Измените его реализацию,  добавив блочную нормализацию.

Когда флаг `normalization` принимает значение "batchnorm" в конструкторе, то необходимо  вставить слой блочной нормализации перед каждой нелинейностью ReLU. Выходы последнего слоя сети не нормализуются. После того, как изменения будут сделаны (возможно Вам повезло и код уже дописан), запустите код ниже для проверки градиента.

СОВЕТ. Возможно, вам будет полезно определить дополнительный вспомогательный слой, аналогичный тому, который содержится в файле `dlcv/layer_utils.py`. Если вы решите это сделать, то сделайте это в файле `dlcv/classifiers/fc_net.py`.

In [ ]:
# Проверка работоспособности полносвязной 3-х слойной сети со слоями блочной нормализации
np.random.seed(231)
N, D, H1, H2, C = 2, 15, 20, 30, 10
X = np.random.randn(N, D)
y = np.random.randint(C, size=(N,))

# Относительные ошибки должны быть в диапазоне 1e-4~1e-10 для W, 
# Относительные ошибки должны быть в диапазоне 1e-08~1e-10 для b,
# Относительные ошибки должны быть в диапазоне  1e-08~1e-09 для beta и gamma.
for reg in [0, 3.14]:
  print('Выполнение проверки с reg = ', reg)
  model = FullyConnectedNet([H1, H2], input_dim=D, num_classes=C,
                            reg=reg, weight_scale=5e-2, dtype=np.float64,
                            normalization='batchnorm')

  loss, grads = model.loss(X, y)
  print('Начальные потери: ', loss)

  for name in sorted(grads):
    f = lambda _: model.loss(X, y)[0]
    grad_num = eval_numerical_gradient(f, model.params[name], verbose=False, h=1e-5)
    print('%s относительная ошибка: %.2e' % (name, rel_error(grad_num, grads[name])))
  if reg == 0: print()

##  Блочная нормализация для глубоких сетей
Выполните код ниже, чтобы обучить шестислойную сеть на подмножестве из 1000 примеров обучения как с блочной нормализацией, так и без нее.

In [ ]:
np.random.seed(231)

# Обучим глубокую нейросеть с использованием блочной нормализации в слоях
hidden_dims = [100, 100, 100, 100, 100]

num_train = 1000
small_data = {
  'X_train': data['X_train'][:num_train],
  'y_train': data['y_train'][:num_train],
  'X_val': data['X_val'],
  'y_val': data['y_val'],
}

weight_scale = 2e-2
# 1. Модель сети  с блочной нормализацией
bn_model = FullyConnectedNet(hidden_dims, weight_scale=weight_scale, normalization='batchnorm')
# 2. Модель сети без нормализации
model = FullyConnectedNet(hidden_dims, weight_scale=weight_scale, normalization=None)

print('Обучение сети c BN')
bn_solver = Solver(bn_model, small_data,
                num_epochs=10, batch_size=50,
                update_rule='adam',
                optim_config={
                  'learning_rate': 1e-3,
                },
                verbose=True,print_every=20)
bn_solver.train()

print('Обучение сети без BN')
solver = Solver(model, small_data,
                num_epochs=10, batch_size=50,
                update_rule='adam',
                optim_config={
                  'learning_rate': 1e-3,
                },
                verbose=True, print_every=20)
solver.train()

Выполните код ячейки ниже, чтобы визуализировать результаты для двух вышеперечисленных сетей. Вы должны обнаружить, что использование блочной нормализации помогает сети сходиться намного быстрее.

In [ ]:
def plot_training_history(title, label, baseline, bn_solvers, plot_fn, bl_marker='.', bn_marker='.', labels=None):
    """функция для построения графиков истории обучения"""
    plt.title(title)
    plt.xlabel(label)
    bn_plots = [plot_fn(bn_solver) for bn_solver in bn_solvers]
    bl_plot = plot_fn(baseline)
    num_bn = len(bn_plots)
    for i in range(num_bn):
        label='с нормал.'
        if labels is not None:
            label += str(labels[i])
        plt.plot(bn_plots[i], bn_marker, label=label)
    label='без нормал.'
    if labels is not None:
        label += str(labels[0])
    plt.plot(bl_plot, bl_marker, label=label)
    plt.legend(loc='lower center', ncol=num_bn+1) 

    
plt.subplot(3, 1, 1)
plot_training_history('Потери обчения','Итерации', solver, [bn_solver], \
                      lambda x: x.loss_history, bl_marker='o', bn_marker='o')
plt.subplot(3, 1, 2)
plot_training_history('Точность обучения','Эпохи', solver, [bn_solver], \
                      lambda x: x.train_acc_history, bl_marker='-o', bn_marker='-o')
plt.subplot(3, 1, 3)
plot_training_history('Валидационная точность','Эпохи', solver, [bn_solver], \
                      lambda x: x.val_acc_history, bl_marker='-o', bn_marker='-o')

plt.gcf().set_size_inches(15, 15)
plt.show()

## Блочная нормализация и инициализация

Проведем небольшой эксперимент для изучения взаимодействия блочной нормализации и инициализации весов.

Первая ячейка будет обучать 8-слойные сети как с бочной нормализацией, так и без нее, используя разные масштабы для инициализации весов. Вторая ячейка будет отображать точность обучения, точность валидации и потери обучения в зависимости от масштаба инициализации весов.

In [ ]:
np.random.seed(231)

# Обучим глубокую нейросеть с использованием блочной нормализации в слоях
# Проведем эксперименты с разными масштабами инициализации весов сети
hidden_dims = [50, 50, 50, 50, 50, 50, 50]
num_train = 1000
small_data = {
  'X_train': data['X_train'][:num_train],
  'y_train': data['y_train'][:num_train],
  'X_val': data['X_val'],
  'y_val': data['y_val'],
}

bn_solvers_ws = {}
solvers_ws = {}
# формируем массив масштабов весов инициализации 
weight_scales = np.logspace(-4, 0, num=20)
# обучаем сети с разными масштабами весов инициализации  
for i, weight_scale in enumerate(weight_scales):
  print('Обучение с weight scale %d / %d' % (i + 1, len(weight_scales)))
  bn_model = FullyConnectedNet(hidden_dims, weight_scale=weight_scale, normalization='batchnorm')
  model = FullyConnectedNet(hidden_dims, weight_scale=weight_scale, normalization=None)
# 1. Модель сети  с блочной нормализацией
  bn_solver = Solver(bn_model, small_data,
                  num_epochs=10, batch_size=50,
                  update_rule='adam',
                  optim_config={
                    'learning_rate': 1e-3,
                  },
                  verbose=False, print_every=200)
  bn_solver.train()
  bn_solvers_ws[weight_scale] = bn_solver
# 2. Модель сети  без блочной нормализации
  solver = Solver(model, small_data,
                  num_epochs=10, batch_size=50,
                  update_rule='adam',
                  optim_config={
                    'learning_rate': 1e-3,
                  },
                  verbose=False, print_every=200)
  solver.train()
  solvers_ws[weight_scale] = solver

In [ ]:
# Построение графиков с результатами экспериментов при разных масштабах весов инициализации 
best_train_accs, bn_best_train_accs = [], []
best_val_accs, bn_best_val_accs = [], []
final_train_loss, bn_final_train_loss = [], []

for ws in weight_scales:
  best_train_accs.append(max(solvers_ws[ws].train_acc_history))
  bn_best_train_accs.append(max(bn_solvers_ws[ws].train_acc_history))
  
  best_val_accs.append(max(solvers_ws[ws].val_acc_history))
  bn_best_val_accs.append(max(bn_solvers_ws[ws].val_acc_history))
  
  final_train_loss.append(np.mean(solvers_ws[ws].loss_history[-100:]))
  bn_final_train_loss.append(np.mean(bn_solvers_ws[ws].loss_history[-100:]))
  
plt.subplot(3, 1, 1)
plt.title('Лучшая валидационная точность vs масштаб весов инициализации')
plt.xlabel('Масштаб веса инициализации')
plt.ylabel('Лучшая валидационная точность')
plt.semilogx(weight_scales, best_val_accs, '-o', label='без нормализации')
plt.semilogx(weight_scales, bn_best_val_accs, '-o', label='с BN нормализацией')
plt.legend(ncol=2, loc='lower right')

plt.subplot(3, 1, 2)
plt.title('Лучшая точность обучения vs масштаб весов инициализации')
plt.xlabel('Масштаб веса инициализации')
plt.ylabel('Лучшая точность обучения')
plt.semilogx(weight_scales, best_train_accs, '-o', label='без нормализации')
plt.semilogx(weight_scales, bn_best_train_accs, '-o', label='c BN нормализацией')
plt.legend()

plt.subplot(3, 1, 3)
plt.title('Финальные потери обучения vs масштаб весов инициализации')
plt.xlabel('Масштаб веса инициализации')
plt.ylabel('Финальные потери обучения')
plt.semilogx(weight_scales, final_train_loss, '-o', label='без нормализации')
plt.semilogx(weight_scales, bn_final_train_loss, '-o', label='c BN нормализацией')
plt.legend()
plt.gca().set_ylim(1.0, 3.5)

plt.gcf().set_size_inches(15, 15)
plt.show()

## Вопрос 1:
Опишите результаты этого эксперимента. Как масштаб веса инициализации  влияет на модели с / без блочной нормализации и почему?

## Ответ:


## Блочная нормализация и размер блока
Проведем небольшой эксперимент для изучения взаимосвязи блочной нормализации и размера блока.

Первая ячейка будет обучать 6-слойные сети как с блочной нормализацией, так и без нее с использованием разных размеров блоков. Вторая -  будет отображать точность обучения и точность валидации  с течением времени.

In [ ]:
def run_batchsize_experiments(normalization_mode):
    # Функция обучает глубокие FC нейросети без слоев нормализации и со слоями BN
    # при использованием миниблоков разных размеров
    np.random.seed(231)
    hidden_dims = [100, 100, 100, 100, 100]
    num_train = 1000
    small_data = {
      'X_train': data['X_train'][:num_train],
      'y_train': data['y_train'][:num_train],
      'X_val': data['X_val'],
      'y_val': data['y_val'],
    }
    
    n_epochs=10              # число эпох
    weight_scale = 2e-2
    batch_sizes = [5,10,50]  # размеры миниблоков
    lr = 10**(-3.5)          # скорость обучения
    solver_bsize = batch_sizes[0]

    print('Без нормализации: размер блока = ',solver_bsize)
    model = FullyConnectedNet(hidden_dims, weight_scale=weight_scale, normalization=None)
    solver = Solver(model, small_data,
                    num_epochs=n_epochs, batch_size=solver_bsize,
                    update_rule='adam',
                    optim_config={
                      'learning_rate': lr,
                    },
                    verbose=False)
    solver.train()
    
    bn_solvers = []
    for i in range(len(batch_sizes)):
        b_size=batch_sizes[i]
        print('С нормализацией: размер блока = ',b_size)
        bn_model = FullyConnectedNet(hidden_dims, weight_scale=weight_scale, normalization=normalization_mode)
        bn_solver = Solver(bn_model, small_data,
                        num_epochs=n_epochs, batch_size=b_size,
                        update_rule='adam',
                        optim_config={
                          'learning_rate': lr,
                        },
                        verbose=False)
        bn_solver.train()
        bn_solvers.append(bn_solver)
        
    return bn_solvers, solver, batch_sizes

batch_sizes = [5,10,50]
bn_solvers_bsize, solver_bsize, batch_sizes = run_batchsize_experiments('batchnorm')

In [ ]:
plt.subplot(2, 1, 1)
plot_training_history('Точность обучения (Блочная нормализация)','Эпохи', solver_bsize, bn_solvers_bsize, \
                      lambda x: x.train_acc_history, bl_marker='-^', bn_marker='-o', labels=batch_sizes)
plt.subplot(2, 1, 2)
plot_training_history('Валидационная точность (Блочная нормализация)','Эпохи', solver_bsize, bn_solvers_bsize, \
                      lambda x: x.val_acc_history, bl_marker='-^', bn_marker='-o', labels=batch_sizes)

plt.gcf().set_size_inches(15, 10)
plt.show()

## Вопрос  2:
Опишите результаты этого эксперимента. Как влияет размер блока на точность обучения/валидации? Почему это происходит?

## Ответ:


# Нормализация на слое

Блочной нормализация  упрощает  обучение сетей, но зависимость от размера блока делает ее менее полезной в сложных сетях, которые имеют ограничение на размер входного блока  (например, из-за ограничений аппаратного обеспечения).

Для смягчения этой проблемы было предложено несколько альтернатив блочной нормализации; одним из таких методов является нормализация на слое [4]. Вместо нормализации в пределах мини-блока (т.е. с находим среднее каждого признака по количеству примеров в блоке) мы выполняем нормализацию по признакам (т.е. используем усреднение по признакам) . Другими словами, при использовании Layer Normalization каждый вектор признаков, соответствующий одной точке данных, нормируется на основе суммы всех членов внутри этого вектора.

[4] [Ba, Jimmy Lei, Jamie Ryan Kiros, and Geoffrey E. Hinton. "Layer Normalization." stat 1050 (2016): 21.](https://arxiv.org/pdf/1607.06450.pdf)

## Вопрос 3:
Какой из  этапов предварительной обработки данных аналогичен нормализации блока, а какой - нормализации на слое?

1. Масштабирование каждого изображения множества данных таким образом, что каналы RGB для каждоц строки пикселей в изображении в сумме дают 1.
2. Масштабирование каждого изображения множества данных таким образом, что каналы RGB для всех пикселей в изображении в сумме дают 1.
3. Вычитание среднего изображения набора данных из каждого изображения в наборе данных.
4. Установка всех значений RGB в 0 или 1 в зависимости от заданного порогового значения.

## Ответ:


# Нормализация на слое: реализация

Теперь реализуете нормализацию на слое. Этот шаг должен быть относительно прост, так как концептуально реализация почти идентична реализации блочной нормализации. Одно из существенных различий заключается в том, что для нормализации на слое мы не отслеживаем скользящие средние, и этап тестирования идентичен этапу обучения, где среднее и дисперсия вычисляются непосредственно для каждого примера входных данных.

Вот что вам нужно сделать:

    В `dlcv/layers.py` реализуете прямой путь для нормализации на слое  в функции `layernorm_forward`.

Запустите ячейку ниже, чтобы проверить свои результаты.

    В `dlcv/layers.py` реализуйте обратный путь для нормализации на слое функции `layernorm_backward`.

Запустите вторую ячейку ниже, чтобы проверить свои результаты.

    Измените `dlcv/classifiers/fc_net.py`, чтобы добавить нормализацию на слое в `FullyConnectedNet`. Когда флаг `normalization` установлен в `"layernorm"` в конструкторе, вы должны вставить код нормализации на слое перед каждой нелинейностью   ReLU.

Запустите третью ячейку ниже, чтобы выполнить эксперимент с размером блока при нормализации на слое.

In [ ]:
# Проверим прямое распростанение в режиме обучения, 
# вычислив средние значения и стандартные отклонения
# прзнаков  до и после нормализации слоя

# Моделирование прямого распространения для двухслойной сети
np.random.seed(231)
N, D1, D2, D3 =4, 50, 60, 3
X = np.random.randn(N, D1)
W1 = np.random.randn(D1, D2)
W2 = np.random.randn(D2, D3)
a = np.maximum(0, X.dot(W1)).dot(W2)

print('Перед нормализацией на слое:')
print_mean_std(a,axis=1)

gamma = np.ones(D3)
beta = np.zeros(D3)

# Средние должны быть близки к нулю, а стандартные отклонения к единице
print('После нормализации на слое (gamma=1, beta=0)')
a_norm, _ = layernorm_forward(a, gamma, beta, {'mode': 'train'})
print_mean_std(a_norm,axis=1)

gamma = np.asarray([3.0,3.0,3.0])
beta = np.asarray([5.0,5.0,5.0])

# Теперб средние должны быть близки к beta, а стандартные отклонения к gamma
print('После нормализации на слое (gamma=', gamma, ', beta=', beta, ')')
a_norm, _ = layernorm_forward(a, gamma, beta, {'mode': 'train'})
print_mean_std(a_norm,axis=1)

In [ ]:
# Проверка градиентов для  нормализации на слое
np.random.seed(231)
N, D = 4, 5
x = 5 * np.random.randn(N, D) + 12
gamma = np.random.randn(D)
beta = np.random.randn(D)
dout = np.random.randn(N, D)

ln_param = {}
fx = lambda x: layernorm_forward(x, gamma, beta, ln_param)[0]
fg = lambda a: layernorm_forward(x, a, beta, ln_param)[0]
fb = lambda b: layernorm_forward(x, gamma, b, ln_param)[0]

dx_num = eval_numerical_gradient_array(fx, x, dout)
da_num = eval_numerical_gradient_array(fg, gamma.copy(), dout)
db_num = eval_numerical_gradient_array(fb, beta.copy(), dout)

_, cache = layernorm_forward(x, gamma, beta, ln_param)
dx, dgamma, dbeta = layernorm_backward(dout, cache)

# Относительная ошибка должна быть в диапазоне 1e-12 и 1e-8
print('Ошибка в dx : ', rel_error(dx_num, dx))
print('Ошибка в dgamma: ', rel_error(da_num, dgamma))
print('Ошибка в dbeta: ', rel_error(db_num, dbeta))

# Нормализация на слое и размер блока

Теперь мы проведем эксперимент с размером блока и нормализацией на слое. По сравнению с предыдущим экспериментом вы должны увидеть заметно меньшее влияние размера блока на историю обучения!

In [ ]:
ln_solvers_bsize, solver_bsize, batch_sizes = run_batchsize_experiments('layernorm')

plt.subplot(2, 1, 1)
plot_training_history('Точность обучения (Нормализация на слое)','Эпохи', solver_bsize, ln_solvers_bsize, \
                      lambda x: x.train_acc_history, bl_marker='-^', bn_marker='-o', labels=batch_sizes)
plt.subplot(2, 1, 2)
plot_training_history('Валидационная точность (Нормализация на слое)','Эпохи', solver_bsize, ln_solvers_bsize, \
                      lambda x: x.val_acc_history, bl_marker='-^', bn_marker='-o', labels=batch_sizes)

plt.gcf().set_size_inches(15, 10)
plt.show()

## Вопрос 4:
Когда нормализация на слое, вероятно, не будет работать хорошо и почему?

1. При использовании в очень глубокой сети
2. При наличии очень небольшого числа признаков
3. При высоком уровне регуляризации


## Ответ:

